In [22]:
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.2 MB 5.6 MB/s eta 0:00:04
   -- ------------------------------------- 1.3/18.2 MB 4.2 MB/s eta 0:00:05
   ---- ----------------------------------- 1.8/18.2 MB 3.6 MB/s eta 0:00:05
   ---- ----------------------------------- 2.1/18.2 MB 3.2 MB/s eta 0:00:06
   ----- ---------------------------------- 2.6/18.2 MB 2.6 MB/s eta 0:00:06
   ------ --------------------------------- 2.9/18.2 MB 2.5 MB/s eta 0:00:07
   ------ --------------------------------- 3.1/18.2 MB 2.3 MB/s eta 0:00:07
   -------- ------------------------------- 3.7/18.2 MB 2.2 MB/s eta 0:00:07
   -------- ------------------------------- 3.9/18.2 MB 2.2 MB/s eta 0:00:07
   --------- ------------------------------ 4.5/18.2 MB 2.2 MB/s eta 0:00:07
   ---------- ----------------------------- 5.0/18.2 MB 2.2 MB/s eta 0:00:06
   --

In [2]:
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer

In [3]:
import os

In [4]:
anime_data = pd.read_csv("anime_dataset.csv")

In [5]:
anime_data.info()
print(anime_data.columns.tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28858 entries, 0 to 28857
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   mal_id                     28858 non-null  int64  
 1   url                        28858 non-null  object 
 2   approved                   28858 non-null  bool   
 3   title                      28858 non-null  object 
 4   title_english              12435 non-null  object 
 5   title_japanese             28737 non-null  object 
 6   title_synonyms             14266 non-null  object 
 7   image_jpg_url              28858 non-null  object 
 8   image_jpg_small_url        28858 non-null  object 
 9   image_jpg_large_url        28858 non-null  object 
 10  image_webp_url             28858 non-null  object 
 11  image_webp_small_url       28858 non-null  object 
 12  image_webp_large_url       28858 non-null  object 
 13  trailer_youtube_id         5553 non-null   obj

In [6]:
columns = ["mal_id", "title", "image_jpg_url", "image_jpg_large_url", "type", "genres","themes","score", "synopsis", "year"]

In [7]:
data = anime_data[columns].copy()

In [9]:
data[data["year"] > 2015].head()

,mal_id,title,image_jpg_url,image_jpg_large_url,type,genres,themes,score,synopsis,year
125,33746,Anitore! XX: Hitotsu Yane no Shita de,https://cdn.myanimelist.net/images/anime/10/82...,https://cdn.myanimelist.net/images/anime/10/82...,TV,Sports,NaN,5.57,Sequel of Anitore! EX. \n\nNew member Sakura I...,2016.0
141,33771,Cheating Craft,https://cdn.myanimelist.net/images/anime/9/818...,https://cdn.myanimelist.net/images/anime/9/818...,TV,"Action, Comedy",School,5.38,In a world where academic success decides your...,2016.0
144,33775,Soul Buster,https://cdn.myanimelist.net/images/anime/3/822...,https://cdn.myanimelist.net/images/anime/3/822...,TV,Action,Samurai,5.73,"Keishuu, China.\n\nThe history books say that ...",2016.0
151,39617,Yakusoku no Neverland 2nd Season,https://cdn.myanimelist.net/images/anime/1815/...,https://cdn.myanimelist.net/images/anime/1815/...,TV,"Mystery, Suspense","Psychological, Survival",5.25,"Emma, Ray, and the rest of the older children ...",2021.0
154,39620,Hakata Mentai! Pirikarako-chan,https://cdn.myanimelist.net/images/anime/1562/...,https://cdn.myanimelist.net/images/anime/1562/...,TV,"Comedy, Gourmet, Slice of Life",NaN,5.13,A daily eating life happening at a street that...,2019.0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28858 entries, 0 to 28857
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mal_id               28858 non-null  int64  
 1   title                28858 non-null  object 
 2   image_jpg_url        28858 non-null  object 
 3   image_jpg_large_url  28858 non-null  object 
 4   type                 28777 non-null  object 
 5   genres               22914 non-null  object 
 6   themes               17020 non-null  object 
 7   score                18735 non-null  float64
 8   synopsis             23750 non-null  object 
 9   year                 6220 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 2.2+ MB


## Filling NaN values

In [9]:
data["type"] = data["type"].fillna("none_type") 

In [10]:
data["genres"] = data["genres"].fillna("none")

In [11]:
data["themes"] = data["themes"].fillna("none")

In [12]:
data["year"] = data["year"].fillna(9999)

In [13]:
df = data.where(data["year"] > 2010)

In [14]:
df = df.dropna()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15971 entries, 1 to 28856
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mal_id               15971 non-null  float64
 1   title                15971 non-null  object 
 2   image_jpg_url        15971 non-null  object 
 3   image_jpg_large_url  15971 non-null  object 
 4   type                 15971 non-null  object 
 5   genres               15971 non-null  object 
 6   themes               15971 non-null  object 
 7   score                15971 non-null  float64
 8   synopsis             15971 non-null  object 
 9   year                 15971 non-null  float64
dtypes: float64(3), object(7)
memory usage: 1.3+ MB


In [16]:
data.shape

(28858, 10)

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28858 entries, 0 to 28857
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mal_id               28858 non-null  int64  
 1   title                28858 non-null  object 
 2   image_jpg_url        28858 non-null  object 
 3   image_jpg_large_url  28858 non-null  object 
 4   type                 28858 non-null  object 
 5   genres               28858 non-null  object 
 6   themes               28858 non-null  object 
 7   score                18735 non-null  float64
 8   synopsis             23750 non-null  object 
 9   year                 28858 non-null  float64
dtypes: float64(2), int64(1), object(7)
memory usage: 2.2+ MB


### Recomendation

In [18]:
class AnimeRecommender:
    def __init__(self, df=None, save_path=None):
        """
        Инициализация: либо из датафрейма, либо из сохраненных данных
        """
        if save_path and os.path.exists(save_path) and df is None:
            self._load_from_disk(save_path)
        elif df is not None:
            self._build_from_scratch(df)
            if save_path:
                self._save_to_disk(save_path)
        else:
            raise ValueError("Нужно указать либо df, либо save_path")

    def _build_from_scratch(self, df):
        """Создаем систему с нуля"""
        print("Создаем систему рекомендаций с нуля...")
        self.df = df.reset_index(drop=True)
        self.model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
        self.index = None
        self._build_index()

    def _build_index(self):
        """Построение FAISS индекса"""
        valid_mask = self.df['synopsis'].notna()
        self.valid_indices = self.df[valid_mask].index.tolist()

        descriptions = self.df.loc[valid_mask, 'synopsis'].tolist()
        print(f"Кодируем {len(descriptions)} описаний...")
        embeddings = self.model.encode(descriptions)

        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)

        embeddings = embeddings.astype('float32')
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)
        print(f"Индекс построен! Векторов: {self.index.ntotal}, валидных индексов: {len(self.valid_indices)}")

    def _save_to_disk(self, save_path):
        """Сохраняем систему на диск"""
        print(f"Сохраняем систему в {save_path}...")

        os.makedirs(save_path, exist_ok=True)
        faiss.write_index(self.index, f"{save_path}/index.faiss")
        self.df.to_parquet(f"{save_path}/data.parquet")

        print("Сохранение завершено!")

    def _load_from_disk(self, save_path):
        """Загружаем систему с диска"""
        print(f"Загружаем систему из {save_path}...")

        self.index = faiss.read_index(f"{save_path}/index.faiss")
        self.df = pd.read_parquet(f"{save_path}/data.parquet")
        self.model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

        print("Загрузка завершена!")

    def recommend_by_title(self, title, k=5):
        mask = self.df['title'].str.lower() == title.lower()

        if not mask.any():
            return [f"Аниме '{title}' не найдено"]

        anime_idx = int(mask[mask].index[0])
        return self._get_recommendations(anime_idx, k)

    def recommend_by_synopsis(self, synopsis, k=5):
        embeddings = self.model.encode([synopsis])

        embeddings = embeddings.astype('float32')
        faiss.normalize_L2(embeddings)

        distances, indices = self.index.search(embeddings, k)

        results = []
        for i, idx in enumerate(indices[0]):
            original_idx = self.valid_indices[idx]
            row = self.df.iloc[original_idx]
            results.append({
                "title": row["title"],
                "similarity": float(distances[0][i]),
                "genres": row.get("genres", "")
            })

        return results

    def _get_recommendations(self, query_idx, k):
        query_vector = self.index.reconstruct(query_idx).reshape(1, -1)
        distances, indices = self.index.search(query_vector.astype('float32'), k + 1)

        results = []
        for i, idx in enumerate(indices[0]):
            if idx != query_idx:
                row = self.df.iloc[idx]
                results.append({
                    'title': row['title'],
                    'similarity': float(distances[0][i]),
                    'genres': row.get('genres', '')
                })
        return results

In [20]:
anm = AnimeRecommender(df=df, save_path="C:/Desktop/Desktop")

Создаем систему рекомендаций с нуля...
Кодируем 15971 описаний...
Индекс построен! Векторов: 15971, валидных индексов: 15971
Сохраняем систему в C:/Desktop/Desktop...
Сохранение завершено!


In [30]:
description = "Мисуми Макото был обычным подростком, которого внезапно призвали в другой мир в качестве «героя». Но богиня этого мира назвала его уродливым, лишила звания героя и сослала в самый дальний уголок своего мира. В этих пустошах он встречает драконов, пауков, орков, дворфов и множество других нечеловеческих рас. У Макото обнаруживаются способности к магии и военному ремеслу, которых у него не могло быть в родном мире. Его ждёт множество встреч, но сумеет ли он выжить в этом новом мире?"

In [31]:
describe_eng = "People is a strong young man who leads the ordinary life of a high school student. One day, in order to save his friends from being attacked by curses, he eats the finger of a Two-faced ghost, which becomes a part of his soul. From that moment on, he shares his body with him. Under the supervision of Satoru Gojo, one of the strongest magicians, Itadori enters a magic college, where they teach how to fight curses."

In [32]:
anm.recommend_by_synopsis(description)

[{'title': 'Tsuki ga Michibiku Isekai Douchuu',
  'similarity': 0.7019407749176025,
  'genres': 'Action, Adventure, Comedy, Fantasy'},
 {'title': 'Munou na Nana',
  'similarity': 0.6984963417053223,
  'genres': 'Suspense'},
 {'title': 'Haiyore! Nyaruko-san',
  'similarity': 0.6754025220870972,
  'genres': 'Comedy, Sci-Fi'},
 {'title': 'Hagure Yuusha no Aesthetica',
  'similarity': 0.6716626882553101,
  'genres': 'Action, Fantasy, Ecchi'},
 {'title': 'Sayonara no Asa ni Yakusoku no Hana wo Kazarou',
  'similarity': 0.6681832075119019,
  'genres': 'Drama, Fantasy'}]